# How does LaminDB's `select` relate to SQLModel's `select`?

In [ ]:
import lamindb as ln
import sqlmodel as sqm
import lnschema_core as core
from datetime import datetime
import pandas as pd

ln.nb.header()

In [ ]:
engine = ln.settings.instance.db_engine()

In [ ]:
def to_df(select_list):
    return pd.DataFrame(
        [row.dict() for row in select_list], columns=select_list[0].__fields__
    )

## Selecting a whole table

### (A) SQLModel

In [ ]:
with sqm.Session(engine) as session:
    df = to_df(session.exec(sqm.select(core.dobject)).all())

In [ ]:
df

### (B) LaminDB

In [ ]:
df = ln.db.select.dobject().df()

In [ ]:
df

### (C) Prisma

```
df = prisma.dobject.findMany()
```

### (F) Declarative

```
df = _.dobject.df()
```

## Subsetting a selection with `where`: equality

### (A) SQLModel

In [ ]:
with sqm.Session(engine) as session:
    df = to_df(
        session.exec(
            sqm.select(core.dobject).where(core.dobject.suffix == ".feather")
        ).all()
    )

In [ ]:
df

### (B) LaminDB

In [ ]:
ln.db.select.dobject(suffix=".feather").df()

### (C) Prisma

```
df = prisma.dobject.findMany(
    {
        "where": {"suffix": ".feather"},
    }
)

df = prisma.dobject.findMany(
    {
        "where": {
            "suffix": {"equals": ".feather"},
        },
    }
)
```

### (D)

```
df = _.dobject.where.suffix(equals=".feather").select()
```

```
df = _.dobject.suffix(equals=".feather").select()
```

```
df = _.dobject.where(_.dobject.suffix.equals(".feather")).select()
```

### (E) Pandas

```
df = _.dobject[_.dobject.suffix == ".feather"]
```

```
df = _.dobject.loc[_.dobject.suffix == ".feather"]
```

```
df = _.dobject.loc[lambda d: d.suffix == ".feather"]
```

### (F) Declarative

```
df = _.dobject.filter(_.dobject.suffix.equals(".feather")).df()
```

```
df = _.dobject.filter(_.dobject.suffix == ".feather").df()
```

## Queries involving linked tables

### (A) SQLModel

In [ ]:
ln.db.select.user().df()

In [ ]:
user_id = "DzTjkKse"

Select version without joins, but using the `in_` operator.

In [ ]:
with sqm.Session(engine) as session:
    # all notebooks authored by the user
    jupynbs = session.exec(
        sqm.select(core.jupynb).where(core.jupynb.created_by == user_id)
    ).all()
    jupynb_ids = [jupynb.id for jupynb in jupynbs]
    # all dtransforms linked to these notebooks
    dtransforms = session.exec(
        sqm.select(core.dtransform).where(core.dtransform.jupynb_id.in_(jupynb_ids))
    ).all()
    dtransform_ids = [dtransform.id for dtransform in dtransforms]
    # all dobjects linked to these dtransforms
    df = to_df(
        session.exec(
            sqm.select(core.dobject).where(
                core.dobject.dtransform_id.in_(dtransform_ids)
            )
        ).all()
    )

In [ ]:
df

### (B) LaminDB

In [ ]:
dobjects_in = ln.db.select.dobject(where=dict(user=dict(id=user_id)))

### (C) Prisma

```
df = prisma.jupynbs.findMany(
    {
        "where": {"created_by": user_id},
        "include": {
            "dtransforms": {
                "include": {"dobject": True},
            },
        },
    }
)
```

### (D)

```
df = _.jupynbs.where.created_by(equals=user_id).dtransforms.dobject.select(
    _.jupynbs.dtransforms.dobject
)
```

```
df = _.jupynbs.where.created_by(equals=user_id).dtransforms.dobject.select(_.dobject)
```

```
df = _.jupynbs.created_by(equals=user_id).dtransforms.dobject.select(_.dobject)
```

### (F) Declarative

```
df = _.user.filter(_.user.created_by.equals(user_id)).dobject.df()
```

### LaminDB

In [ ]:
# ln.db.select.dobject(where=dict(user=dict(id="DzTjkKse")))

## Subsetting a selection with `where`: general comparisons

### (A) SQLModel

Subsetting the previous select to the most recent additions.

In [ ]:
time = datetime(2022, 10, 11, 23, 0, 0)

In [ ]:
with sqm.Session(engine) as session:
    df = to_df(
        session.exec(
            sqm.select(core.dobject).where(
                core.dobject.suffix == ".feather", core.dobject.created_at > time
            )
        ).all()
    )

In [ ]:
df

### (B) LaminDB

We can't do this right now.

### (C) Prisma

```
df = prisma.dobject.findMany(
    {
        "where": {"suffix": ".feather", "created_at": {"gt": time}},
    }
)
```

### (D)

```
df = _.dobject.where.suffix(equals=".feather").created_at(gt=time).select()
```

### (F) Declarative

```
df = _.dobject \
    .filter(AND(_.dobject.suffix.equals(".feather), _.dobject.suffix.gt(time))) \
    .df()
```

## Subsetting a selection with `where`: combine expressions using `or`

SQLModel only.

### (C) Prisma

```
df = prisma.dobject.findMany(
    {
        "where": {"OR": {[{"suffix": ".feather"}, {"suffix": ".csv"}]}},
    }
)
```

### (D)

```
df = _.dobject.where(
    OR(_.dobject.suffix.equals(".feather"), _.dobject.suffix.equals(".csv"))
).select()
```

```
df = _.dobject.where(
    OR(_.dobject.suffix(".feather"), _.dobject.suffix(".csv"))
).select()
```

### (F)

```
df = _.dobject \
    .filter(OR(_.dobject.suffix.equals(".feather"), _.dobject.suffix.equals(".csv"))) \
    .df()
```

## Subsetting a selection with `where`: limit select results

SQLModel only.

## Limiting and offsetting select results

This is important for selecting tables with very high numbers of rows.

SQLModel only.

### (C) Prisma

```
df = prisma.dobject.findMany(
    {
        "skip": 3,
        "take": 4,
    }
)
```

### (D)

```
df = _.dobject.select(skip=3, take=4)
```

### (E) Pandas

`df = _.dobject[4:7]`

```
df = _.dobject.iloc[4:7]
```

### (F)

`df = _.dobject.slice(4, 7).df()`

## A more complex example

```
df = db \
    .user \
    .filter(db.user.created_at.equals(date)) \
    .order_by(db.user.handle) \
    .dobject \
    .filter(db.dobject.suffix.equals(".csv"))
    .select(db.dobject.columns, db.dobject.dtransform.columns)
    .slice(0, 1)
    .one()
```

```
df = (
    # Start block
    db 

    # Table block 1 
    .user               
    .filter()
    .order_by()
    .slice()

    # Table block 2 
    .dobject    
    .order_by()
    .slice()   
    .filter()
    
    # Table block 3
    .dtransform               
    .filter()
    .order_by()
    .slice()

    # Result block
    .select()           
    .one() # .maybe_one()
    .df() # .object()
)
```